# SCORING FUNCTION

In [ ]:
# Needs to be run to make sure there is no version mismatch
!pip install scikit-learn==1.0.2

In [1]:
import sklearn as sc

In [2]:
sc.__version__

'1.3.0'

In [18]:
def project_2_scoring(data):
    from sklearn.preprocessing import OneHotEncoder
    from copy import deepcopy
    from sklearn.linear_model import LogisticRegression
    import pickle
    import category_encoders as ce
    import numpy as np
    from sklearn.preprocessing import StandardScaler
    # we deep copy original data
    X_train= data.copy()
    #replacing values in RevLineCr with Y and N
    X_train['RevLineCr']=X_train['RevLineCr'].replace('0','N')
    X_train['RevLineCr']=X_train['RevLineCr'].replace('1','Y')
    X_train['RevLineCr']=X_train['RevLineCr'].replace('T','Y')
    #replacing values in RevLineCr with null that have no context
    X_train['RevLineCr'] = X_train['RevLineCr'].apply(lambda x: x if x in ['Y', 'N'] else np.nan)

    
    X_train['RevLineCr'].fillna(X_train['RevLineCr'].mode()[0], inplace=True)

    #replacing values in LowDoc with Y and N
    X_train['LowDoc']=X_train['LowDoc'].replace('0','N')
    X_train['LowDoc']=X_train['LowDoc'].replace('S','Y')
    X_train['LowDoc']=X_train['LowDoc'].replace('A','N')
    
    X_train['LowDoc'] = X_train['LowDoc'].apply(lambda x: x if x in ['Y', 'N'] else np.nan)


    X_train['LowDoc'].fillna(X_train['LowDoc'].mode()[0], inplace=True)

    #extract index column 
    index=X_train[['index']]
    X_train.drop('index',inplace=True,axis=1,errors='ignore')
    cat=[]
    num=[]
    colu=[]
    for col in X_train.columns:
        if X_train[col].isna().any() == True:
            colu.append(col)
    num=[col for col in colu if (X_train[col].dtype != 'object') ] 
    cat=[col for col in colu if (X_train[col].dtype == 'object') ]
    #imputing null values
    for i in cat:
        X_train[i].fillna(X_train[i].mode().iloc[0],inplace=True)
    for i in num:
        X_train[i].fillna(X_train[i].median(),inplace=True)
    # Fearure engineering
    # Feature 1: Loan Size Category
    X_train['LoanSizeCategory'] = pd.cut(X_train['GrAppv'], bins=[0, 50000, 150000, 350000, 1000000, 5000000], labels=['Micro', 'Small', 'Medium', 'Large', 'Very Large'])

        # Feature 2: Job Creation Category
    X_train['JobCreationCategory'] = pd.cut(X_train['CreateJob'] + X_train['RetainedJob'], bins=[-1, 0, 5, 10, 20, 50, 100, float('inf')], labels=['None', 'Few', 'Some', 'Moderate', 'Many', 'A Lot', 'Massive'])

        # Feature 3: High-Risk Industry (NAICS codes starting with 72 are typically hospitality, considered higher risk)
    X_train['HighRiskIndustry'] = X_train['NAICS'].apply(lambda x: 1 if str(x).startswith('72') else 0)

        # Feature 4: Loan/Grant Ratio
    X_train['LoanGrantRatio'] = X_train['GrAppv'] / X_train['SBA_Appv']

        # Feature 5: Per Employee Investment
    X_train['PerEmployeeInvestment'] = X_train['GrAppv'] / (X_train['NoEmp'] + 1)  # Avoid division by zero



        # Feature 6: Business Age - New business flag (assuming NewExist codes: 1 for existing, 2 for new)
    X_train['NewBusiness'] = X_train['NewExist'].apply(lambda x: 1 if x == 2 else 0)

        # Feature 7: Urban Business - Flag for businesses in urban areas (assuming UrbanRural codes: 1 for urban, 2 for rural, 0 for undefined)
    X_train['UrbanBusiness'] = X_train['UrbanRural'].apply(lambda x: 1 if x == 1 else 0)

        # Feature 8: Franchise Flag - Flag for whether the business is a franchise (assuming FranchiseCode: 1 or 0 indicates not a franchise, any other value indicates a franchise)
    X_train['FranchiseFlag'] = X_train['FranchiseCode'].apply(lambda x: 0 if x == 1 or x == 0 else 1)

        # Feature 9: Expansion Plan - Flag businesses creating more jobs than retaining (CreateJob > RetainedJob)
    X_train['ExpansionPlan'] = X_train.apply(lambda x: 1 if x['CreateJob'] > x['RetainedJob'] else 0, axis=1)

        # Feature 10: Economic Sector - Extract broader economic sector from NAICS code (first two digits of NAICS code)
    X_train['EconomicSector'] = X_train['NAICS'].apply(lambda x: int(str(x)[:2]) if x > 0 else 0)

    # Feature 11: Credit Utilization Ratio
    X_train['CreditUtilizationRatio'] = X_train['GrAppv'] / X_train['SBA_Appv']

# Feature 12: Profitability Indicator (Profitability as Retained Jobs greater than Created Jobs)
    X_train['ProfitabilityIndicator'] = (X_train['RetainedJob'] > X_train['CreateJob']).astype(int)

# Feature 13: Seasonal Adjustment (Seasonal industries, e.g., Retail in Q4 - Using NAICS)
    seasonal_naics = [44, 45]  # Retail trade NAICS codes
    X_train['SeasonalAdjustment'] = X_train['NAICS'].apply(lambda x: 1 if int(str(x)[:2]) in seasonal_naics else 0)

# Feature 14: Geographic Region (Categorizing ZIP codes into regions, assuming arbitrary division for example)
    X_train['GeographicRegion'] = pd.cut(X_train['Zip'], bins=4, labels=['North', 'South', 'East', 'West'])

# Feature 15: Loan Size per Employee
    X_train['LoanPerEmployee'] = X_train['GrAppv'] / (X_train['NoEmp'] + 1)  # Adding 1 to avoid division by zero

    for col in X_train.select_dtypes(include=['category']).columns:
        X_train[col] = X_train[col].astype('object')
    # Identify categorical variables with less than 10 unique values for one-hot encoding
    cat_vars_for_one_hot = [col for col in X_train.columns if (X_train[col].nunique() < 10) and (X_train[col].dtype == 'object')]

    # Now, for target and WOE encoding, identify the remaining categorical variables
    remaining_cat_vars = [col for col in X_train.columns if (X_train[col].dtype == 'object') and (col not in cat_vars_for_one_hot)]

    numeric=[col for col in X_train.columns if (X_train[col].dtype != 'object') ]

    #importing trained encoders/scalers
    with open('woe_encoder.pkl', 'rb') as file:
        loaded_woe_encoder = pd.read_pickle(file)
    with open('standardscaler.pkl', 'rb') as file:
        loaded_scaler = pd.read_pickle(file)
    with open('onehot_encoder.pkl', 'rb') as file:
        loaded_onehot_encoder = pd.read_pickle(file)
    drops=['NewExist','CreateJob','RetainedJob','NAICS','GrAppv','SBA_Appv']
    res = [item for item in numeric if item not in drops]
    
    X_train = X_train.join(loaded_woe_encoder.transform(X_train[remaining_cat_vars]).add_suffix('_woe'))
    
    X_train[res] = loaded_scaler.transform(X_train[res])
    
    X_train.drop(columns=remaining_cat_vars,axis=1,inplace=True,errors='ignore')
    X_train.drop(columns=drops,axis=1,inplace=True,errors='ignore')

    X_train_ohe = loaded_onehot_encoder.transform(X_train[cat_vars_for_one_hot])
    # Convert to DataFrame and name columns
    columns = loaded_onehot_encoder.get_feature_names_out(cat_vars_for_one_hot)
    X_train_ohe = pd.DataFrame(X_train_ohe, columns=columns, index=X_train.index)

    X_train = pd.concat([X_train.drop(columns=cat_vars_for_one_hot), X_train_ohe], axis=1)
    #importing trained logreg model
    with open('xgb_model.pkl', 'rb') as file:
        loaded_model = pd.read_pickle(file)
    y_pred_proba = loaded_model.predict_proba(X_train)
    y_pred = (y_pred_proba[:,1] > 0.63).astype(np.int16)
    d = {"index":index.to_numpy().ravel(),
             "label":y_pred,
             "probability_0":y_pred_proba[:,0],
             "probability_1":y_pred_proba[:,1]}

    return pd.DataFrame(d)

In [20]:
import pandas as pd
data = pd.read_csv('SBA_loans_project_2_holdout_students_valid.csv')
x=project_2_scoring(data)
x

,index,label,probability_0,probability_1
0,0,0,0.786404,0.213596
1,1,0,0.672249,0.327751
2,2,0,0.695789,0.304211
3,3,0,0.872023,0.127977
4,4,0,0.728167,0.271833
...,...,...,...,...
99803,99803,1,0.342779,0.657221
99804,99804,0,0.806731,0.193269
99805,99805,0,0.871434,0.128566
99806,99806,0,0.706304,0.293696


# KAGGLE 

In [23]:
x=x[['index','probability_1']]
x.rename(columns={'index': 'ID'}, inplace=True)


/var/folders/x3/5n0yyb7911zg532jx71xlkrr0000gn/T/ipykernel_56916/1532254769.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.rename(columns={'index': 'ID'}, inplace=True)


In [25]:
x

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/formatters.py:343: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()


,ID,probability_1
0,0,0.213596
1,1,0.327751
2,2,0.304211
3,3,0.127977
4,4,0.271833
...,...,...
99803,99803,0.657221
99804,99804,0.193269
99805,99805,0.128566
99806,99806,0.293696


In [27]:
x.to_excel('submission.xlsx', index=False)

TypeError: write() got an unexpected keyword argument 'engine_kwargs'